### Манифест

Выделяется структура из текстовых документов.
Необходимо для каждого параграфа документа определить его тип и уровень вложенности относительно всего документа. 

На вход последовательно подается список параграфов документа. Для каждого параграфа необходимо определить его тип: заголовок, список, текст или другое, а также пометить увеличивается ли уровень вложенности документа, не меняется или уменьшается, если уменьшается или увеличивается, то насколько.

#### 1. Заголовок

Заголовок всего документа обычно располагается в начале документа, подзаголовки, как правило, начинают новую главу, подглаву, секцию и т. д. Может располагаться по центру, выделяться жирным шрифтом, курсивом и т. д.

#### 2. Список

Пронумерованная (числами, буквами или специальными маркерами) последовательность строк. Начинается с нумерации и как правило, располагается рядом с аналогично пронумерованными строками.

#### 3. Текст

Все остальные строки, содержащие текст.

#### Об уровнях вложенности

Изначальный уровень вложенности документа - 0. То есть первый заголовок документа имеет вложенность 0, так же как и любой текст до этого заголовка. Текст, следующий после этого заголовка (относящийся к данному заголовку), имеет уровень вложенности, больший на 1. Элементы списков имеют уровень вложенности, зависящий от того, куда они вкладываются. Если список является частью текстового блока, то он имеет такой же уровень, как и этот текстовый блок. Если список вложен в другой список, то его уровень вложенности увеличивается на 1.

При понижении уровня необходимо смотреть, продолжением какого списка или к заголовку какого уровня относится очередной параграф документа.

Каждому параграфу нужно сопоставить два числа - номер типа параграфа (либо 0, если другое) и уровень вложенности

* Идея для более удобной разметки: отображать docx документ на странице, выделяя строку, которую нужно пометить, цветом. Можно отображать рядом текст размеченных строк и их метки. После разметки целого документа добавить возможность переразметки. [(Может пригодиться)](https://www.cyberforum.ru/html/thread1762308.html)

* корректность работы docx-парсера???

* составление дерева на основе уровней и типов

In [84]:
from document_parser import DOCXParser
import os
import json
import docx
import re
import numpy as np
import xgboost as xgb
from sklearn.metrics import f1_score

In [50]:
# document = docx.Document('examples/train_examples/0.docx')
# document.paragraphs[0].runs[0].font.color.rgb = docx.shared.RGBColor(0xff, 0x99, 0xcc)

In [85]:
# [{"name", "paragrahps": [{"text", "type", "level"}]}]
# type - header - 1, list - 2, text - 3
# numeration begins with 0
results = []

for i in range(1, 7):
    path = f"examples/train_examples/{i}.docx"
    document = docx.Document(path)
    item = {"name": path}
    item["paragraphs"] = []
    end = False
    while not end:
        for paragraph in document.paragraphs:
            if not p:
                continue
            p = {"text": paragraph.text}
            print(paragraph.text)
            type_ = input("type: ")
            level = input("level: ")
            p["type"] = type_
            p["level"] = level
            item["paragraphs"].append(p)
        answer = input("end?")
        if answer == "y":
            end = True
            results.append(item)

In [2]:
with open('examples/train_examples/labeled.json', "r") as f:
    data = json.load(f)

In [3]:
clear_data = []

for item in data:
    new_item = {"name": item["name"], "paragraphs": []}
    for p in item['paragraphs']:
        p_info = {}
        if p['text'].strip():
            p_info['text'] = p['text'].strip()
            p_info['type'] = int(p['type'])
            p_info['level'] = int(p['level'])
            new_item["paragraphs"].append(p_info)
    if new_item["paragraphs"]:
        clear_data.append(new_item)

In [4]:
clear_data[3]['paragraphs'] = clear_data[3]['paragraphs'][:4]
clear_data[0]

{'name': 'examples/train_examples/1.docx',
 'paragraphs': [{'text': 'Договор № ____', 'type': 1, 'level': 0},
  {'text': 'на выполнение комплекса работ и услуг по содержанию объектов (зданий и строений), сервисному и техническому обслуживанию комплекса объектов инженерной инфраструктуры',
   'type': 1,
   'level': 0},
  {'text': 'г. Москва \t\t\t\t\t\t\t\t«___»________2015 года',
   'type': 3,
   'level': 1},
  {'text': 'Общество с ограниченной ответственностью «Объединенная дирекция по управлению активами и сервисами Центра разработки и коммерциализации новых технологий (инновационного центра «Сколково»)» (ООО «ОДАС Сколково»), именуемое в дальнейшем «Заказчик», в лице Директора Дирекции по эксплуатации объектов недвижимости  Троценко Дениса Сергеевича, действующего на основании Доверенности № 37 от 10.07.2015 г., с одной стороны, и',
   'type': 3,
   'level': 1},
  {'text': '______________________________________(__)_, именуемое в дальнейшем «Исполнитель», в лице ____________________

In [54]:
data_without_labels = []
labels = []

for item in clear_data:
    new_item = {"name": item["name"], "paragraphs": []}
    for p in item['paragraphs']:
        new_item["paragraphs"].append(p["text"])
        labels.append((p["type"], p["level"]))
    data_without_labels.append(new_item)

#### Признаки

* жирность, курсив, подчеркивание, размер, выравнивание, отступ с помощью DOCXParser
* число символов в первом слове
* число цифр в первом слове
* длина строки
* начало с тире, цифры, буквы и т. п.
* те же признаки для предыдущего и следующего параграфов
* наличие стиля heading, type из DOCXParser

In [68]:
alignment2number = {
    "left": 0,
    "right": 1,
    "center": 2,
    "both": 3
}

type2number = {
    "paragraph": 1,
    "list_item": 2,
    "raw_text": 3
}


def find_paragraph(paragraph_list, text):
    for p in paragraph_list:
        # TODO correct comparison
#         print(f"real text={' '.join(p['text'].split())}")
#         print(f"text={' '.join(text.split())}")
#         print("===========")
        if " ".join(p["text"].split()).find(" ".join(text.split())) != -1:
            result = p.copy()
            paragraph_list.remove(p)
            return result
    return None
#     raise KeyError(f"paragraph {text} not found in the document {paragraph_list}")
    

def extract_annotations(annotations):
    result = {"indent": [0, 0, 0, 0], "alignment": 0, "size": 0, "bold": 0, "italic": 0, "underlined": 0}
    if not annotations:
        return result
    for start, end, annotation in annotations:
        if annotation.startswith("indent"):
            d = json.loads(re.sub("'", '"', annotation[7:]))
            result["indent"] = [d["left"], d["start"], d["hanging"], d["firstLine"]]
        elif annotation.startswith("alignment"):
            result["alignment"] = alignment2number[annotation[10:]]
        elif annotation.startswith("size"):
            result["size"] = int(annotation[5:])
        else:
            for item in ["bold", "italic", "underlined"]:
                if annotation.startswith(item):
                    result[item] = 1
                    break
    return result
            

def extract_doc_features(docs_info):
    """
    docs_info = list of {"name", "paragraphs": ["text of 1 line", "text of 2 line"]}
    returns list of features for each paragraph
    """
    features = []
    for doc_info in docs_info:
        path = doc_info["name"]
        parser = DOCXParser(path)
        lines_info = parser.get_lines_with_meta()
        for p in doc_info["paragraphs"]:
            p_features = []
            p_info = find_paragraph(lines_info, p)
            if p_info:
                p_features.append(type2number[p_info["type"]])
            else:
                p_features.append(3)
                p_info = {"text": "", "annotations": None}
            p_annotations = extract_annotations(p_info["annotations"])
            p_features += p_annotations["indent"]
            p_features += [p_annotations["alignment"], p_annotations["size"], 
                          p_annotations["bold"], p_annotations["italic"], p_annotations["underlined"]]
            p_features.append(len(p_info["text"].split()))
            
            if p_info["text"].split():
                first_word = p_info["text"].split()[0]
            else:
                first_word = ""
            p_features.append(len(first_word))
            p_features.append(len(re.findall(r"\d+", first_word)))
            features.append(p_features)
    return features

In [69]:
features = extract_doc_features(data_without_labels)

In [73]:
features = np.array(features)

In [77]:
features.shape, len(labels)

((130, 13), 130)

In [81]:
y = [y*10 + x for x, y in labels]

In [79]:
clf = xgb.XGBClassifier()

In [82]:
clf.fit(features, y)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=0, num_parallel_tree=1,
              objective='multi:softprob', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=None, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [89]:
f1_score(test_y, clf.predict(test_features), average='macro')

1.0